In [ ]:
import cv2
import tkinter as tk

import datetime
import os

from math import degrees, radians
from pprint import pprint

import numpy as np
import pydicom

from skimage.io import imread
from skimage.draw import circle, line
from skimage.color import gray2rgb, rgb2gray

from scipy.fftpack import fft, ifft, fftfreq

import matplotlib.pyplot as plt


In [ ]:
# def choose_file(window_title):
#     root = Tk()
#     filename = askopenfilename(initialdir="..", title=window_title,
#                                filetypes=(("jpeg files", ".jpg"), ("all files", ".")))
#     root.destroy()
#     return filename


# filename = choose_file("wybierz zdjecie")
filename = ""
img = cv2.imread(filename, 0)
img = cv2.resize(img,(512,512))
img = cv2.normalize(img,None,0,255,cv2.NORM_MINMAX)
R=img.shape[0]/2